In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable

In [2]:
def to_var(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)

def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0,1)

# Image processing
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(.5,.5,.5), std=(.5,.5,.5))])

In [3]:
# datasets, dataloader
dsets = datasets.MNIST(root='data/', train=True, download=True, transform=transform)


Processing...
Done!


In [8]:
dsets_loader = torch.utils.data.DataLoader(dataset=dsets, batch_size=128, shuffle=True)

In [9]:
# Discriminator
D = nn.Sequential(nn.Linear(784, 256), 
                  nn.LeakyReLU(.2), 
                  nn.Linear(256, 256), 
                  nn.LeakyReLU(.2), 
                  nn.Linear(256, 1), 
                  nn.Sigmoid())

# Generator
G = nn.Sequential(nn.Linear(64, 256), 
                  nn.LeakyReLU(.2), 
                  nn.Linear(256, 256), 
                  nn.LeakyReLU(.2), 
                  nn.Linear(256, 784), 
                  nn.Tanh())

if torch.cuda.is_available():
    D.cuda()
    G.cuda()

In [10]:
# binary loss entropy and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=.0003)
g_optimizer = torch.optim.Adam(G.parameters(), lr=.0003)

In [14]:
import os

os.mkdir('data/images')

In [15]:
# Training

for epoch in range(200):
    for i, (images, _) in enumerate(dsets_loader):
        # build mini-batch datasets
        batch_size = images.size(0)
        images = to_var(images.view(batch_size, -1))
        
        # create the labels which are later used for input for loss computation
        real_labels = to_var(torch.ones(batch_size))
        fake_labels = to_var(torch.zeros(batch_size))
        
        #========== Train the discriminator ============#
        # compute BCELoss using real images where BCELoss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # second term of the loss is always 0 since real_labels == 1 : - y * log(D(x))
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # compute BCELoss using fake images
        # first term of the loss is always 0 since fake_labels == 0 : - (1-y) * log(1 - D(x))
        z = to_var(torch.randn(batch_size, 64))
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # backprop, optimize
        d_loss = d_loss_real + d_loss_fake
        D.zero_grad()
        d_loss.backward()
        d_optimizer.step()
        
        #========== Train the generator ============#
        # compute loss using fake images
        z = to_var(torch.randn(batch_size, 64))
        fake_images = G(z)
        outputs = D(fake_images)
        
        # train G to maximize log(D(G(z))) instead of minimizing log(1 - D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)
        
        # backprop, optimizer
        D.zero_grad()
        G.zero_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 300 == 0:
            print("Epoch: {0:3d}/200,".format(epoch+1), 
                  "Iterations: {0:4d},".format(i+1), 
                  "d_loss: {:.4f},".format(d_loss.data[0]), 
                  "g_loss: {:.4f},".format(g_loss.data[0]), 
                  "D(x): {:.2f},".format(real_score.data.mean()), 
                  "D(G(z)): {:.2f}".format(fake_score.data.mean()))
    
    # save real images
    if (epoch + 1) == 1:
        images = images.view(images.size(0), 1, 28, 28)
        save_image(denorm(images.data), 'data/images/real_images.png')
        
    # save sampled images
    fake_images = fake_images.view(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images.data), 'data/images/fake_images_%d.png' % (epoch+1))

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/functional.py:1189: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Epoch:   1/200, Iterations:  300, d_loss: 0.4402, g_loss: 2.4957, D(x): 0.93, D(G(z)): 0.26


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/functional.py:1189: UserWarning: Using a target size (torch.Size([96])) that is different to the input size (torch.Size([96, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Epoch:   2/200, Iterations:  300, d_loss: 0.4921, g_loss: 2.7280, D(x): 0.80, D(G(z)): 0.13
Epoch:   3/200, Iterations:  300, d_loss: 0.3390, g_loss: 3.3649, D(x): 0.85, D(G(z)): 0.10
Epoch:   4/200, Iterations:  300, d_loss: 1.5264, g_loss: 1.9026, D(x): 0.58, D(G(z)): 0.33
Epoch:   5/200, Iterations:  300, d_loss: 0.8980, g_loss: 1.6728, D(x): 0.71, D(G(z)): 0.29
Epoch:   6/200, Iterations:  300, d_loss: 2.0298, g_loss: 1.6152, D(x): 0.56, D(G(z)): 0.40
Epoch:   7/200, Iterations:  300, d_loss: 0.5688, g_loss: 2.0982, D(x): 0.83, D(G(z)): 0.24
Epoch:   8/200, Iterations:  300, d_loss: 0.6221, g_loss: 3.5193, D(x): 0.81, D(G(z)): 0.22
Epoch:   9/200, Iterations:  300, d_loss: 0.9517, g_loss: 2.0178, D(x): 0.75, D(G(z)): 0.34
Epoch:  10/200, Iterations:  300, d_loss: 0.8668, g_loss: 1.8030, D(x): 0.83, D(G(z)): 0.37
Epoch:  11/200, Iterations:  300, d_loss: 0.7450, g_loss: 2.4188, D(x): 0.78, D(G(z)): 0.29
Epoch:  12/200, Iterations:  300, d_loss: 0.5217, g_loss: 2.0539, D(x): 0.85, D(

KeyboardInterrupt: 

In [ ]:
torch.save(G.state.dict(), 'generator.pkl')
torch.save(D.state.dict(), 'discriminator.pkl')